# How to build a search engine for Wikipedia


In this article we try to build a simple full text search engine for Wikipedia. There are already enterprise search and indexing options like ElasticSearch and Apache Solr. But we are trying to see how hard would that be to build a search engine starting from scratch.
We could use [Inverted index](https://en.wikipedia.org/wiki/Inverted_index). But for this project we use [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) model to capture the important words in a document. Then using [Hashing Trick](https://en.wikipedia.org/wiki/Feature_hashing) we will build a model of each document with their term frequencies. A similar model to [Document-term Matrix](https://en.wikipedia.org/wiki/Document-term_matrix). 



Apache Spark already implemented TF-IDF vectorization method in their Mllib package. That means this method can be executed in parallel across many computers in a cluster which makes it scalable. I will be running the code on a local machine but it can be run on a cluster as well. One important factor is understanding the number of hash buckets we need to build the tf-idf model. Let's count all the distinct words in the whole Wikipedia and see how many words are there. That could give us a sense of number of hash buckets needed to minimize hash collisions. 

In [ ]:
from operator import add
from pyspark import SparkContext, SparkConf

conf = SparkConf()\
        .setAppName("Wiki_Search_Engine")\
        .setMaster("local[*]")\
        .set("spark.driver.memory", "10g")\
        .set("spark.driver.maxResultSize", "4g")

sc = SparkContext(conf=conf)
rawData = sc.textFile(r"wiki\artile_per_line.txt")
flatten_words = rawData.flatMap(lambda x: x.split('\t')[1].split())
words_joint = flatten_words.map(lambda x: (x, 1)).reduceByKey(add)
print "Number of distinct words in the whole Wikipedia: ", words_joint.count()
# Number of distinct words in the whole Wikipedia:  26764007

Running the above code shows we have 26764007 distinct words in English Wikipedia. Hashing all those words, which most of them might not be very important in a document's content, can be very expensive and might lead to wrong results. So, instead of hashing every word in Wikipedia we are going to hash only meaningful words. There are several steps that are done before feeding the data to Spark and they are listed as:

 * All the stopwords are removed from the document body. TF-IDF model is supposed to help with scoring down stopwords, the words that are very common in a natural language like "the", "of" and etc, but it's just a lot of computational burden so we filter them out beforehand.
 * All the words that are not in a dictionary nor in the set of all Wikipedia titles are removed. Combining Wiki title words with a good dictionary gives us a very large set of meaningful words, name of places and people and so on.
 * All the words that have the same lemma (root) are treated as the same. In perticular we lemmatize the words and take their lemma. This reduces the number of distinct words and helps to use less hash buckets. Words like swimming, swims or swimmed are all the same in the sense that they all carry the same root. 
 * punctuations are removed from the text for obvious reasons

There are many packages for NLP that we could use to apply the steps mentioned above. [NLTK](http://www.nltk.org/) is the most popular in this field. However, NLTK is fairly large with many proxy objects around their models which are not picklable. Spark uses [Cloud Pickle](https://github.com/apache/spark/blob/master/python/pyspark/cloudpickle.py) to serialize the objects created during the run time. I had little luck with integrating NLTK objects with Spark. So, I decided to do the pre-processing without Spark. Using NLTK was very slow since the operations are usually single-threaded and it doesn't harness full power of CPU specially when there are multiple cores sitting there and doing nothing. I found an alternative NLP package for Python called [Spacy](https://spacy.io/) which implements Multi-Threaded algorithems for the expensive parts of the pipeline. It is also designed in a way that it applies the whole pipline at once from tokenizing to parsing, Part of Speach (POS) tagging and lemmatizing. For anyone who wants to try using Spacy I recommend to compile it with OpenMP option enabled otherwise the multi-threading won't work. See [#267](https://github.com/spacy-io/spaCy/issues/267) for example. I managed to compile it on Windows and use it without any issue. Here is the code for doing the pre-processing on Wiki documents using Spacy:  

In [ ]:
import re
from nltk.corpus import stopwords
from spacy.en import English
import nltk
import spacy.attrs
import time
import codecs

fo = open(r'wiki\titles.txt')
raw_text = fo.read()
fo.close()

wiki_title_words = set(raw_text.split())
english_vocab = set(w.lower() for w in nltk.corpus.words.words())
stop_words = set(stopwords.words("english"))

NLP = English()

fo = codecs.open(r'wiki\artile_per_line.txt', 'r', encoding='utf-8') # title \t document
out = codecs.open(r"Wiki\article_lemma.txt", 'w', encoding='utf-8')


for doc in NLP.pipe(fo, n_threads=4):    
    title_words = []
    passed_title = False
    for candidate in doc:
        if '\t' not in candidate.lemma_: # title \t document
            if passed_title:
                if (candidate.lemma_ not in stop_words) and (candidate.pos_ != u'PUNCT') and \
                    (candidate.lemma_ in english_vocab or candidate.lemma_ in wiki_title_words):

                    out.write(candidate.lemma_ + u' ')
            else:
                title_words.append(candidate.orth_)
        else:
            out.write(u''.join(title_words) + candidate.orth_)
            passed_title = True

    out.write('\n')

fo.close()
out.close()

Now let's run the word count in Spark and see how many words we have left now:

In [ ]:
from operator import add
from pyspark import SparkContext, SparkConf

conf = SparkConf()\
        .setAppName("Wiki_Search_Engine")\
        .setMaster("local[*]")\
        .set("spark.driver.memory", "10g")\
        .set("spark.driver.maxResultSize", "4g")

sc = SparkContext(conf=conf)
rawData = sc.textFile(r"wiki\article_lemma.txt")
flatten_words = rawData.flatMap(lambda x: x.split('\t')[1].split())
words_joint = flatten_words.map(lambda x: (x, 1)).reduceByKey(add)
print "Number of distinct lemmas in the whole Wikipedia: ", words_joint.count()
# Number of distinct lemmas in the whole Wikipedia:  561354

Number of distinct words left in the documents after pre-processing is 561354. That means more than 98% of the words from the original documents are all removed. This means we can comfortably hash the remaining words to build our tf-idf model.

In [ ]:
sc = SparkContext(conf=conf)
rawData = sc.textFile(r"wiki\articles_lemma.txt")
documents = rawData.map(lambda line : line.split('\t')[1].split())
titles = rawData.map(lambda line : line.split('\t')[0])
titles.cache()

hashingTF = HashingTF(20000000)  #20 Million hash buckets just to make sure it fits in memory
tf = hashingTF.transform(documents)
idf = IDF(minDocFreq=10).fit(tf)
tfidf = idf.transform(tf)
tfidf.cache()

Now that we have tf-idf model for all documents. We can send queries and return the documents that have the highest score for that perticular word:

In [7]:
QueryTF = hashingTF.transform(["tabriz"])
QueryHashValue = QueryTF.indices[0]
QueryRelevance = tfidf.map(lambda x: x[QueryHashValue])
zippedResults = QueryRelevance.zip(titles)
print "Top 10 related documents:"
for (k, v) in zippedResults.sortByKey(ascending=False).take(10):
    print v

Top 10 related documents:
Tabriz
Azerbaijan(Iran)
Safina-yiTabriz
Tabrizrug
OldAzerilanguage
TabrizKhanate
Persiancarpet
Safavidart
MachineSaziTabrizF.C.
EastAzerbaijanProvince


As you can see it works! Tabriz is a city in Iran and all the returned results are related to the word with the first link to the Wikipedia page of the city. I run this on my local machine which takes a minute or two for each query but if we distribute this on a cluster it should return the results much faster. 